In [4]:
import pandas as pd

In [5]:
df = pd.read_table('SMSSpamCollection.txt', header=None)
df.columns = ['y', 'text']

In [6]:
df['y'] = df.iloc[:, 0].map({'ham': 0, 'spam': 1})

In [7]:
texts = df.text
labels = df.y

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [9]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [30]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), 
                                          TfidfTransformer(), 
                                          clf(max_iter=1000)), 
                          texts, labels, cv=10, scoring='f1').mean())
    print("\n")

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.8776671468367306


<class 'sklearn.svm._classes.LinearSVC'>
0.9504398834735606


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.9501581620848405




In [16]:
clf = Pipeline([("vectorizer", CountVectorizer()),
                ("transformer", TfidfTransformer()),
                ("classifier", LogisticRegression(max_iter=1000))])

In [17]:
clf.fit(texts, labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...
                ('transformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_s

In [18]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
        "FreeMsg: Txt: claim your reward of 3 hours talk time",
        "Have you visited the last lecture on physics?",
        "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
        "Only 99$"]

In [19]:
clf.predict(test)

array([1, 1, 0, 0, 0], dtype=int64)

In [31]:
for ngram in [(2,2), (3,3), (1,3)]:
    clf = Pipeline([("vectorizer", CountVectorizer(ngram_range=ngram)),
                    ("transformer", TfidfTransformer()),
                    ("classifier", LogisticRegression(random_state=2))])
    score = cross_val_score(clf, texts, labels, cv=10, scoring='f1').mean()
    print(f'{ngram}  : {score:.3f}') 

(2, 2)  : 0.452
(3, 3)  : 0.220
(1, 3)  : 0.804
